In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/train.csv')
data.head()

,client_id,treatment_flg,purchased
0,ad6561e2d8,1,1
1,7c1ccbf93f,1,1
2,b58fadcab6,1,1
3,e99e6fabb9,0,0
4,27fb6f8520,1,1


In [3]:
clients = pd.read_csv('data/clients.csv').drop(columns=['client_id.1'])
clients.head()

,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
2,00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
3,0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
4,00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U


In [4]:
purchase = pd.read_csv('data/train_purch_hist.csv')
purchase.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


In [5]:
products = pd.read_csv('data/products.csv')
products.head()

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.400,0,0
1,0003870676,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,acd3dd483f,10486c3cf0,0.680,0,0
2,0003ceaf69,c3d3a8e8c6,f2333c90fb,419bc5b424,f6148afbc0,271.0,f597581079,764e660dda,0.500,0,0
3,000701e093,ec62ce61e3,4202626fcb,88a515c084,48cf3d488f,172.0,54a90fe769,03c2d70bad,0.112,0,0
4,0007149564,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,63417fe1f3,f329130198,0.600,0,0


# Каково значение аплифта на всем трейне?

In [6]:
uplift = data.loc[data['treatment_flg']==1, 'purchased'].mean() - data.loc[data['treatment_flg']==0, 'purchased'].mean()
uplift

0.03354516795221374

# Выберите правильные утверждения:

Последняя покупка в train_purch была совершена в марте 2018 года. (False)

In [7]:
from datetime import date


dates = purchase['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0])).to_numpy()
dates.max()

datetime.date(2019, 3, 18)

Аплифт в группе людей, возраст которых больше 50 лет, примерно в два раза выше чем у остальных людей.(True)

In [8]:
df = data.merge(clients[['client_id', 'age']], on='client_id', how='left')
df1 = df[df.age > 50]
df2 = df[df.age <= 50]
uplift1 = df1.loc[df1['treatment_flg']==1, 'purchased'].mean() - df1.loc[df1['treatment_flg']==0, 'purchased'].mean()
uplift2 = df2.loc[df2['treatment_flg']==1, 'purchased'].mean() - df2.loc[df2['treatment_flg']==0, 'purchased'].mean()
uplift1, uplift2

(0.0506140212921522, 0.021301489366581716)

В трейне мужчин больше чем женщин. (False)

In [9]:
df = data.merge(clients[['client_id', 'gender']], on='client_id', how='left')
df[df.gender=='M'].count()[0] > df[df.gender=='F'].count()[0]

False

Медианный возраст всех людей в трейне равен 39 годам.  

In [10]:
df = data.merge(clients[['client_id', 'age']], on='client_id', how='left')
df.age.median()

45.0

# На какую максимальную сумму в train_purch была совершена покупка? (округлить до целого)

In [11]:
purchase.purchase_sum.max()

29611.48

# Каково мат. ожидание кол-ва посетителей во всех магазин в train_purch в день?

In [12]:
df = purchase[['client_id', 'transaction_datetime']].drop_duplicates()
df.transaction_datetime = df.transaction_datetime.apply(lambda date: date.split()[0])
df.groupby(by=['transaction_datetime']).count().mean()

client_id    23855.855932
dtype: float64

In [13]:
tmp = df.groupby(by=['transaction_datetime']).count()
x, p = np.unique(tmp.client_id, return_counts=True)
p = p / p.sum()
(x * p).sum()

23855.85593220339

# Какого значения аплифта можно достичь, если осуществить воздействие на людей, с момента последней покупки которых прошло 10 и более дней?

In [14]:
df = purchase[['client_id', 'transaction_datetime']]
df['transaction_datetime'] = df['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0]))
df = df.groupby('client_id').max()
df = df.rename(columns={'transaction_datetime': 'last_trans'}).reset_index()
df = df.merge(data, on='client_id', how='left')
laste_date = date(2019, 3, 18)
df['more'] = df.last_trans.apply(lambda date: (laste_date - date).days >= 10)

df_ = df[df.more]
uplift = df_.loc[df_['treatment_flg']==1, 'purchased'].mean() - df_.loc[df_['treatment_flg']==0, 'purchased'].mean()
uplift

C:\Users\Андрей\AppData\Local\Temp\3\ipykernel_13788\541311909.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transaction_datetime'] = df['transaction_datetime'].apply(lambda x: date.fromisoformat(x.split()[0]))


0.03724156974927306

# Какова доля продукции собственного производства в покупках среди всех покупок?

In [15]:
df = purchase[['client_id', 'product_id']]
df = df.merge(products[['product_id', 'is_own_trademark']], on='product_id', how='left')
df[df.is_own_trademark == 1].shape[0] / df.shape[0]

0.1523224771222515

# Влияет ли факт покупки самого популярного продукта на аплифт? Если влияет, то каково значение аплифта?

In [16]:
df = purchase[['client_id', 'product_id']]
df = df.groupby(by=['product_id']).count()
df.sort_values(by=['client_id'], ascending=False).head()

,client_id
product_id,
4009f09b04,636488
15ccaa8685,219727
bf07df54e1,144643
3e038662c0,134126
4dcf79043e,129695


In [17]:
def get_buy_popular(client):
    try:
        return (tmp.loc[client, 'product_id'] == '4009f09b04').any()
    except:
        return tmp.loc[client, 'product_id'] == '4009f09b04'


tmp = purchase[['client_id', 'product_id']].set_index('client_id')
df = data.copy()
df['is_buy_popular'] = df['client_id'].apply(get_buy_popular)
df_ = df[df.is_buy_popular]
uplift = df_.loc[df_['treatment_flg']==1, 'purchased'].mean() - df_.loc[df_['treatment_flg']==0, 'purchased'].mean()
uplift

0.026437105461474042

# Вопрос 8
Разделим покупателей на две группы: владеют картой покупателя > 100 дней (группа А), владеют картой покупателя <= 100 дней (группа Б). Правда ли то, что аплифт в группе А меньше, чем в группе Б? Если правда, то на сколько меньше (разница между процентами)?

In [18]:
df = data.merge(clients[['client_id', 'first_issue_date']], on='client_id', how='left')
df['first_issue_date'] = df['first_issue_date'].apply(lambda d: date.fromisoformat(d.split()[0]))
df['group'] = df['first_issue_date'].apply(lambda d: (laste_date - d).days > 100)

df_A = df[df.group]
upliftA = df_A.loc[df_A['treatment_flg']==1, 'purchased'].mean() - df_A.loc[df_A['treatment_flg']==0, 'purchased'].mean()

df_B = df[~df.group]
upliftB = df_B.loc[df_B['treatment_flg']==1, 'purchased'].mean() - df_B.loc[df_B['treatment_flg']==0, 'purchased'].mean()

upliftA, upliftB, upliftA < upliftB, upliftB - upliftA

(0.031109531100951826, 0.05470674738360215, True, 0.023597216282650324)

# Вопрос 9
Разделим покупателей на две группы: покупали продукты собственного производства хоть раз (группа А), вообще не покупали продукты собственного производства (группа Б). Правда ли то, что аплифт в группе А меньше чем в группе Б? Если правда, то на сколько меньше? (округлить до среднего)

In [19]:
df = purchase[['client_id', 'product_id']]
df = df.merge(products[['product_id', 'is_own_trademark']], on='product_id', how='left')
df = df.groupby(by=['client_id']).sum('is_own_trademark')
df = df.reset_index()
df = data.merge(df, on='client_id', how='left')
df['group']= df.is_own_trademark.apply(lambda x: x > 0)

In [20]:
df_A = df[df.group]
upliftA = df_A.loc[df_A['treatment_flg']==1, 'purchased'].mean() - df_A.loc[df_A['treatment_flg']==0, 'purchased'].mean()

df_B = df[~df.group]
upliftB = df_B.loc[df_B['treatment_flg']==1, 'purchased'].mean() - df_B.loc[df_B['treatment_flg']==0, 'purchased'].mean()

upliftA, upliftB, upliftA < upliftB, upliftB - upliftA

(0.03174733889332593, 0.07902768028809476, True, 0.04728034139476883)

# Вопрос 10
Разделим покупателей на две группы. В группе А у большей части покупок клиента чек больше среднего чека по всей выборке. В группе Б у  большей части покупок клиента чек меньше среднего чека по всей выборке. Правда ли то, что аплифт в группе А меньше, чем в группе Б?

In [21]:
df = purchase[['client_id', 'transaction_id', 'purchase_sum']].drop_duplicates('transaction_id')
mean = df.purchase_sum.mean()
df = df.set_index('client_id')
df_main = data.copy()

In [22]:
df_main['group'] = df_main['client_id'].apply(lambda client: 
                                              (df.loc[client, 'purchase_sum'] > mean)
                                              .sum() > df.loc[client].shape[0] / 2)

In [23]:
df_A = df_main[df_main.group]
upliftA = df_A.loc[df_A['treatment_flg']==1, 'purchased'].mean() - df_A.loc[df_A['treatment_flg']==0, 'purchased'].mean()

df_B = df_main[~df_main.group]
upliftB = df_B.loc[df_B['treatment_flg']==1, 'purchased'].mean() - df_B.loc[df_B['treatment_flg']==0, 'purchased'].mean()

upliftA, upliftB, upliftA < upliftB, upliftB - upliftA

(0.030065406049781362, 0.035237203184547705, True, 0.005171797134766343)